In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rahulhavalad/minor-prjt")

print("Path to dataset files:", path)

In [ ]:
pip install tensorflow==2.14.0

In [1]:
import tensorflow as tf
import os

# Enable GPU dynamic memory growth (for local environments)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 16
EPOCHS = 40  # Allow higher max epochs; EarlyStopping will halt early

TRAIN_DIR = '/kaggle/input/minor-prjt/Cereal_Crop_Disease_Split/Train'
VALIDATION_DIR = '/kaggle/input/minor-prjt/Cereal_Crop_Disease_Split/Validation'
TEST_DIR = '/kaggle/input/minor-prjt/Cereal_Crop_Disease_Split/Test'

LABELS = [
    'Corn_Blight', 'Corn_Common_Rust', 'Corn_Greay_Leaf_Spot', 'Corn_Healthy',
    'Rice_Bacterial_Blight', 'Rice_Blast', 'Rice_Brown_Spot',
    'Sorghum_Anthracnose_Red_Rot', 'Sorghum_Cereal_Grain_Molds',
    'Sorghum_Head_Smut', 'Sorghum_Loose_Smut', 'Sorghum_Rust',
    'Wheat_Brown_Rust', 'Wheat_Healthy', 'Wheat_Septoria', 'Wheat_Yellow_Rust'
]

# Preprocessing
def preprocess_data(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def load_dataset(directory, shuffle=True):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='int',
        class_names=LABELS,
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=shuffle
    )
    return dataset.map(preprocess_data).prefetch(tf.data.AUTOTUNE)

# Load datasets
train_dataset = load_dataset(TRAIN_DIR, shuffle=True)
val_dataset = load_dataset(VALIDATION_DIR, shuffle=False)
test_dataset = load_dataset(TEST_DIR, shuffle=False)

# Model architecture
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,  # Stop if no improvement in 5 epochs
    restore_best_weights=True,
    verbose=1
)

# Train
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

# Evaluate
loss, accuracy = model.evaluate(test_dataset)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")

# Representative dataset for quantization
def representative_dataset():
    for images, _ in train_dataset.take(100):
        yield [tf.cast(images, tf.float32)]

# Convert to quantized TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save files
os.makedirs('/kaggle/working/', exist_ok=True)

with open('/kaggle/working/cereal_crop_disease_model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

with open('/kaggle/working/labels.txt', 'w') as f:
    for label in LABELS:
        f.write(label + '\n')

print("🚀 Model training completed. Quantized model and labels saved.")


2025-05-26 10:03:39.873455: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-26 10:03:39.873510: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-26 10:03:39.873571: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 10844 files belonging to 16 classes.
Found 1348 files belonging to 16 classes.
Found 1371 files belonging to 16 classes.
Epoch 1/40
339/339 [==============================] - 259s 748ms/step - loss: 1.0753 - accuracy: 0.6585 - val_loss: 0.4797 - val_accuracy: 0.8561
Epoch 2/40
339/339 [==============================] - 247s 726ms/step - loss: 0.4627 - accuracy: 0.8525 - val_loss: 0.3316 - val_accuracy: 0.9088
Epoch 3/40
339/339 [==============================] - 270s 794ms/step - loss: 0.3307 - accuracy: 0.8941 - val_loss: 0.2802 - val_accuracy: 0.9177
Epoch 4/40
339/339 [==============================] - 253s 743ms/step - loss: 0.2725 - accuracy: 0.9138 - val_loss: 0.2368 - val_accuracy: 0.9318
Epoch 5/40
339/339 [==============================] - 267s 786ms/step - loss: 0.2295 - accuracy: 0.9319 - val_loss: 0.2223 - val_accuracy: 0.9303
Epoch 6/40
339/339 [==============================] - 269s 791ms/step - loss: 0.2027 - accuracy: 0.9396 - val_loss: 0.1964 - val_accuracy: 0.94

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


🚀 Model training completed. Quantized model and labels saved.


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
